
# Agno Teams — Operations Deep Dive (Storage → Custom Loggers)

A practical, concept-first walkthrough of essential Agno Teams operations from **Storage** through **Custom Loggers**.  
Diagrams use **Mermaid** code blocks; many notebook renderers (including GitHub and some Jupyter viewers) display them directly.

**What’s inside**
- Storage — persisting sessions & state
- Memory — capturing and recalling user facts
- Knowledge — using a knowledge base and filters
- Pre‑hooks & Post‑hooks — validate/transform input & output
- Guardrails — safety patterns using hooks
- Metrics — understand cost/time usage
- Cancelling a Run — stop long executions
- Custom Loggers — route logs to your own handlers

---



## Table of Contents
1. [Storage](#storage)
2. [Memory](#memory)
3. [Knowledge](#knowledge)
4. [Pre‑hooks & Post‑hooks](#pre-post-hooks)
5. [Guardrails](#guardrails)
6. [Metrics](#metrics)
7. [Cancelling a Run](#cancelling-a-run)
8. [Custom Loggers](#custom-loggers)
9. [Cheat‑sheet](#cheat-sheet)



---
## 1) Storage <a id="storage"></a>

**Goal:** Persist session history and state so a Team can continue across requests/processes.

**Why it matters**
- Teams are stateless by default; storage ties runs together.
- Required for history features, summaries, and stateful workflows.

**Minimal setup (example)** — choose a DB adapter and pass it to `Team`:


In [ ]:

# Minimal example: configure storage (choose a DB backend you actually use)
# from agno.team import Team
# from agno.db.sqlite import SqliteDb
#
# db = SqliteDb(db_file="tmp/teams.db")
# team = Team(members=[], db=db)
# team.print_response("Hello!")            # run 1
# team.print_response("What did I say?")   # run 2 (history is available)



**What gets stored (typical fields)**

| Field | Meaning |
| --- | --- |
| `session_id` | Unique conversation identifier |
| `user_id` | Who the session belongs to |
| `team_id` | Which team handled it |
| `runs` | Run records (messages, tool calls, results) |
| `session_data` / `team_data` | Serialized state/config |
| `summary` | Optional condensed recap |
| `created_at` / `updated_at` | Timestamps |

**Flow**
```mermaid
sequenceDiagram
  autonumber
  participant U as User
  participant T as Team.run()
  participant DB as Storage (DB)
  U->>T: send input
  T->>DB: read/attach session (by session_id)
  T->>T: produce response (delegate/tools)
  T->>DB: append run + update state
  T-->>U: final answer
```



---
## 2) Memory <a id="memory"></a>

**Goal:** Remember user facts/preferences across messages and sessions.

**Key points**
- Enable memories on the Team to capture and recall user details.
- Works best with a database configured.

**Minimal setup (example)**


In [ ]:

# from agno.team import Team
# from agno.db.sqlite import SqliteDb
#
# db = SqliteDb(db_file="tmp/teams.db")
# team = Team(members=[], db=db, enable_user_memories=True)
# team.print_response("My name is Sam.")
# team.print_response("What is my name?")  # should recall "Sam"



**Memory lifecycle**
```mermaid
flowchart LR
  A[User says a fact] --> B{Memory enabled?}
  B -- yes --> C[Team stores fact as memory]
  C --> D[Later run]
  D --> E[Recall relevant memory]
  B -- no --> F[No memory captured]
```



---
## 3) Knowledge <a id="knowledge"></a>

**Goal:** Let the Team search a knowledge base and ground responses on retrieved docs.

**Concepts**
- **Knowledge base**: documents with metadata (type, owner, date, etc.).
- **Filters**: narrow retrieval by metadata (e.g., `project:"phoenix"`).
- **Agentic filtering** (optional): allow the team to infer filters from the question.

**Typical flow**
```mermaid
sequenceDiagram
  participant U as User
  participant TL as Team Leader
  participant KB as Knowledge Base
  U->>TL: Ask question
  TL->>KB: search_knowledge_base(filters?)
  KB-->>TL: top docs/snippets
  TL->>TL: synthesize grounded answer
  TL-->>U: respond with citations/notes
```

**Lightweight pattern (pseudo‑code)**


In [ ]:

# from agno.team import Team
# from agno.agent import Agent
# from agno.models.openai import OpenAIChat
# from agno.tools.knowledge import KnowledgeTools  # hypothetical toolkit
#
# kb_tools = KnowledgeTools()  # exposes search_knowledge_base(...) tool
#
# researcher = Agent(name="KB Researcher", tools=[kb_tools])
# team = Team(
#     members=[researcher],
#     model=OpenAIChat(id="gpt-5-mini"),
#     # enable_agentic_knowledge_filters=True,  # let the leader infer filters
# )
# team.print_response("Summarize our 'contracts' policy for 2024.")



**Design tips**
- Keep documents richly tagged so filters are powerful.
- Add short doc summaries to improve retrieval quality.
- When grounding is critical, instruct the Team to only answer from retrieved docs.



---
## 4) Pre‑hooks & Post‑hooks <a id="pre-post-hooks"></a>

**Purpose**
- **Pre‑hooks** run before the model — validate, sanitize, enrich input.
- **Post‑hooks** run after the model — validate/transform the output.

**At a glance**

| Hook | Runs | Common uses |
| --- | --- | --- |
| Pre‑hook | before `Team.run()` processing | input length checks, PII scrubbing, auth, routing |
| Post‑hook | after response is generated | format/length checks, profanity filtering, audit tagging |

**Flow**
```mermaid
sequenceDiagram
  autonumber
  participant In as Incoming Input
  participant Pre as Pre‑hooks
  participant Team as Team + Members
  participant Post as Post‑hooks
  In->>Pre: validate/transform
  Pre->>Team: safe input
  Team->>Post: candidate output
  Post-->>In: final output to caller
```

**Tiny examples (structure only)**


In [ ]:

# from agno.team import Team
# from agno.exceptions import InputCheckError, OutputCheckError, CheckTrigger
#
# def check_len(run_input, *_, **__):
#     if len(run_input.input_content) > 2000:
#         raise InputCheckError("Input too long", check_trigger=CheckTrigger.INPUT_NOT_ALLOWED)
#
# def guarantee_short(run_output):
#     if run_output.content and len(run_output.content) > 1200:
#         raise OutputCheckError("Output too long", check_trigger=CheckTrigger.INPUT_NOT_ALLOWED)
#
# team = Team(pre_hooks=[check_len], post_hooks=[guarantee_short])



---
## 5) Guardrails <a id="guardrails"></a>

**Idea:** Implement safety and policy checks using hooks and/or tools.

**Typical patterns**
- Block disallowed content (e.g., violent instructions) in a **pre‑hook**.
- Mask secrets/PII before sending to a model.
- Post‑filter outputs for compliance (e.g., remove personal data).

**Decision sketch**
```mermaid
flowchart TD
  A[Incoming request] --> B{Policy OK?}
  B -- no --> C[Reject with safe message]
  B -- yes --> D[Proceed to Team]
  D --> E{Output compliant?}
  E -- no --> F[Redact/Rewrite/Block]
  E -- yes --> G[Return to user]
```

**Tips**
- Keep checks deterministic (regex/allow‑lists) where possible.
- Log violations with enough context for audits (use custom loggers below).



---
## 6) Metrics <a id="metrics"></a>

**What you get**
- Per message, per member, team‑level, and session‑level aggregates.
- Token counts, duration, time‑to‑first‑token, optionally provider‑specific metrics.

**Common fields**

| Field | Meaning |
| --- | --- |
| `input_tokens`, `output_tokens`, `total_tokens` | Text token usage |
| `reasoning_tokens` | If supported by the model |
| `duration` | Total time for the run |
| `time_to_first_token` | Latency to first streamed token |
| `provider_metrics` | Extra counters from LLM vendor |

**Quick inspection (example)**


In [ ]:

# from agno.utils.pprint import pprint_run_response
#
# response = team.run("Short task")
# pprint_run_response(response, markdown=True)  # pretty view
#
# # Or inspect directly:
# print("Team metrics:", response.metrics)
# for msg in (response.messages or []):
#     if msg.role == "assistant":
#         print("Message metrics:", msg.metrics)



---
## 7) Cancelling a Run <a id="cancelling-a-run"></a>

**Use‑case:** Long tasks that should be interruptible (e.g., via API/UI).

**Sequence**
```mermaid
sequenceDiagram
  participant App as App Thread
  participant Team as Team.run(stream=True)
  App->>Team: start run (stream)
  App-->>App: capture run_id from first chunks
  App->>Team: cancel_run(run_id)
  Team-->>App: emits *cancelled* event
```

**Minimal pattern**


In [ ]:

# run_id_box = {}
#
# for ev in team.run("Do a lengthy task", stream=True):
#     if "run_id" not in run_id_box and getattr(ev, "run_id", None):
#         run_id_box["run_id"] = ev.run_id
#     # ... optionally stream content to UI ...
#
# team.cancel_run(run_id_box.get("run_id"))



---
## 8) Custom Loggers <a id="custom-loggers"></a>

**Purpose:** Route Agno logs into your own logging pipeline/format.

**How it fits**
```mermaid
flowchart LR
  A[Agno internals] -->|log events| B[Custom logger]
  B --> C[Console/ELK/SIEM/Cloud logging]
```

**Minimal setup**


In [ ]:

# import logging
# from agno.utils.log import configure_agno_logging, log_info
#
# custom = logging.getLogger("my_team_logger")
# h = logging.StreamHandler()
# h.setFormatter(logging.Formatter("[TEAM] %(levelname)s: %(message)s"))
# custom.addHandler(h)
# custom.setLevel(logging.INFO)
# custom.propagate = False
#
# configure_agno_logging(custom_team_logger=custom)
# log_info("Hello from custom logger!")



---
## Cheat‑sheet <a id="cheat-sheet"></a>

| Topic | One‑liner |
| --- | --- |
| Storage | Attach a DB to persist sessions/history/state. |
| Memory | `enable_user_memories=True` to remember user facts. |
| Knowledge | Add a search tool + (optionally) agentic filters to ground answers. |
| Pre‑hooks | Validate/sanitize input before model work. |
| Post‑hooks | Check/transform output before returning. |
| Guardrails | Implement policy checks using hooks and logging. |
| Metrics | Inspect `TeamRunOutput.metrics` and message metrics for cost/time. |
| Cancel | Stream, capture `run_id`, then `cancel_run(run_id)`. |
| Custom Loggers | `configure_agno_logging(...)` to integrate with your stack. |

---

**Next steps**
- Add a database and turn on memories for continuity.
- Introduce one pre‑hook (input length/PII mask) and one post‑hook (output length).
- Wire your log pipeline and start capturing metrics in dashboards.
